In [2]:
import pandas as pd
df = pd.read_csv("chicago_beach_weather.csv")
df.drop(columns=['Wet Bulb Temperature', 'Humidity', 'Rain Intensity', 'Interval Rain', 'Total Rain', 'Precipitation Type', 'Maximum Wind Speed', 'Barometric Pressure', 'Solar Radiation', 'Heading', 'Battery Life', 'Measurement ID', 'Wind Direction', 'Wind Speed', 'Measurement Timestamp Label'])


,Station Name,Measurement Timestamp,Air Temperature
0,63rd Street Weather Station,12/31/2016 11:00:00 PM,-1.30
1,Foster Weather Station,12/31/2016 11:00:00 PM,-1.56
2,Oak Street Weather Station,12/31/2016 11:00:00 PM,-0.30
3,63rd Street Weather Station,12/31/2016 10:00:00 PM,-0.70
4,Foster Weather Station,12/31/2016 10:00:00 PM,-1.06
...,...,...,...
23681,Foster Weather Station,01/01/2016 01:00:00 AM,-4.56
23682,Oak Street Weather Station,01/01/2016 01:00:00 AM,-3.20
23683,63rd Street Weather Station,01/01/2016 12:00:00 AM,-2.80
23684,Foster Weather Station,01/01/2016 12:00:00 AM,-3.67


In [3]:
from collections import defaultdict
import datetime
unique_stations = set(df['Station Name']) #Extract all unique stations
#we can make a nested dictionary, {'Station Name' -> 'Date' -> 'Start', 'End', 'Low', 'High'}
weather_data = defaultdict(dict)
for station in unique_stations:
    weather_data[station] = defaultdict(list)

for i in range(len(df.index)):
    weather_station, timestamp, temperature = df.iat[i, 0], df.iat[i, 1], df.iat[i, 2]
    time_zone = timestamp[-2:] #extract AM/PM info
    timestamp = datetime.datetime.strptime(timestamp, "%m/%d/%Y %H:%M:%S %p")
    month, day, year = timestamp.month, timestamp.day, timestamp.year
    hour = timestamp.hour
    
    date_key = str(month) + "/" + str(day) + "/" + str(year)
    if date_key not in weather_data[weather_station]:
        weather_data[weather_station][date_key] = [0, 0, float("-inf"), float("inf")]
    if timestamp.hour == 12 and time_zone == "AM":
        weather_data[weather_station][date_key][0] = temperature
    elif timestamp.hour == 11 and time_zone == "PM":
        weather_data[weather_station][date_key][1] = temperature
    max_temperature = weather_data[weather_station][date_key][2]
    min_temperature = weather_data[weather_station][date_key][3]
    if temperature > max_temperature:
        weather_data[weather_station][date_key][2] = temperature
    elif temperature < min_temperature:
        weather_data[weather_station][date_key][3] = temperature

        
    

In [4]:
col_list = ["Station Name", "Date", "Start Temperature", "End Temperature", "Low Temperature", "High Temperature"]
number_of_columns = 365 * len(weather_data)
output_df = pd.DataFrame(columns = col_list, index = range(number_of_columns))
i = 0
for station in weather_data:
    for date in weather_data[station]:
        output_df.iloc[i, output_df.columns.get_loc("Station Name")] = station
        output_df.iloc[i, output_df.columns.get_loc("Date")] = date
        output_df.iloc[i, output_df.columns.get_loc("Start Temperature")] = weather_data[station][date][0]
        output_df.iloc[i, output_df.columns.get_loc("End Temperature")] = weather_data[station][date][1]
        output_df.iloc[i, output_df.columns.get_loc("Low Temperature")] = weather_data[station][date][2]
        output_df.iloc[i, output_df.columns.get_loc("High Temperature")] = weather_data[station][date][3]
        i += 1


In [7]:
output_df.head(50)

,Station Name,Date,Start Temperature,End Temperature,Low Temperature,High Temperature
0,63rd Street Weather Station,12/31/2016,3.5,-1.3,5.6,3.5
1,63rd Street Weather Station,12/30/2016,0.6,3.6,3.6,-2.8
2,63rd Street Weather Station,12/29/2016,2.8,1.1,2.8,0.4
3,63rd Street Weather Station,12/28/2016,-1.9,3.3,7.1,-2.2
4,63rd Street Weather Station,12/27/2016,-0.5,-1.5,-0.3,-2.5
5,63rd Street Weather Station,12/26/2016,5.8,-0.1,12.9,5.8
6,63rd Street Weather Station,12/25/2016,0,4.4,4.4,0.8
7,63rd Street Weather Station,12/24/2016,2.3,1.9,2.8,1.4
8,63rd Street Weather Station,12/23/2016,-0.6,2.2,3.3,-0.6
9,63rd Street Weather Station,12/22/2016,1.5,-0.9,4.7,-0.9


In [6]:
output_df.to_csv("Organized Data")